# Adaptive PDE discretizations on cartesian grids 
## Volume : Divergence form PDEs
## Part : Linear elasticity
## Chapter : Elastic energy

We present a new discretization of the Dirichlet energy arising in linear elasticity, defined by a field of positive definite Hooke tensors, and associated to any small displacement field. The approach is based on Selling's decomposition of the Hooke tensor, and for this reason is only applies in dimension two at the moment.

The Dirichlet energy of linear elasitcity, defined for (small) displacement maps $v : \Omega \to R^d$, reads
$$
    E(v) := \frac 1 2 \int_\Omega \sum_{ijkl} c_{ijkl}(x) \epsilon_{ij}(x) \epsilon_{kl}(x) \ dx,
$$
where the indices $i,j,k,l$ range from $0$ to $d-1$. We denoted by $c_{ijkl}(x)$ the Hooke tensor at a point $x \in \Omega$, and introduced the symmetrized gradient of the displacement field, also known as the strain tensor $\epsilon$
$$
    \epsilon_{ij}(x) = \frac 1 2 \Big (\frac{\partial v_i}{\partial x_j} + \frac{\partial v_j}{\partial x_i} \Big).
$$

**Dimension two only.** Extending the numerical scheme presented in this notebook to dimension three requires an implementation of Voronoi's first reduction (a tool from lattice geometry) in dimension $6$, which is the dimension of the Hooke tensor. This is possible in principle, but it requires substantial work, and it is not currently available in the agd library, which only implements this tool in dimension $\leq 5$.

[**Summary**](Summary.ipynb) of volume Non-Divergence form PDEs, this series of notebooks.

[**Main summary**](../Summary.ipynb) of the Adaptive Grid Discretizations 
	book of notebooks, including the other volumes.

# Table of contents
  * [1. Decomposition of a hooke tensor](#1.-Decomposition-of-a-hooke-tensor)
    * [1.1 Generic tensor](#1.1-Generic-tensor)
    * [1.2 Isotropic tensor](#1.2-Isotropic-tensor)
  * [2. Finite difference energy](#2.-Finite-difference-energy)
    * [2.1 Comparison with automatic differentiation](#2.1-Comparison-with-automatic-differentiation)
    * [2.2 Structure of the mass matrix](#2.2-Structure-of-the-mass-matrix)



**Acknowledgement.** The experiments presented in these notebooks are part of ongoing research, 
some of it with PhD student Guillaume Bonnet, in co-direction with Frederic Bonnans.

Copyright Jean-Marie Mirebeau, University Paris-Sud, CNRS, University Paris-Saclay

## 0. Importing the required libraries

In [1]:
import sys; sys.path.insert(0,"..") # Allow import of agd from parent directory (useless if conda package installed)
#from Miscellaneous import TocTools; print(TocTools.displayTOC('ElasticityDirichlet','NonDiv'))

In [2]:
from agd import LinearParallel as lp
from agd import FiniteDifferences as fd
from agd.Metrics.Seismic import Hooke
from agd import AutomaticDifferentiation as ad
from agd import Domain
from agd.Plotting import savefig; #savefig.dirName = 'Images/ElasticityDirichlet'

norm_infinity = ad.Optimization.norm_infinity
norm_average = ad.Optimization.norm_average
mica,_ = Hooke.mica # Hooke tensor associated to this crystal

In [3]:
import numpy as np; xp=np; allclose=np.allclose
import matplotlib.pyplot as plt
from copy import copy
import scipy.sparse; import scipy.sparse.linalg

In [4]:
def ReloadPackages():
    from Miscellaneous.rreload import rreload
    global lp,fd,Hooke,ad,Domain
    lp,fd,Hooke,ad,Domain = rreload([lp,fd,Hooke,ad,Domain],rootdir="../..")
    ad.array.caster = xp.asarray

### 0.1 Additional configuration

Uncomment the following line to run the notebook on the GPU. (This is purely for compatibility testing, as no intensive computation is involved.)

In [5]:
xp,mica,allclose = map(ad.cupy_friendly,(xp,mica,allclose))

Replacing numpy with cupy, set to output 32bit ints and floats by default.
Using cp.asarray(*,dtype=np.float32) as the default caster in ad.array.
Replacing ndarray members with their cupy variants, for object of type <class 'agd.Metrics.Seismic.hooke.Hooke'>
Setting float32 compatible default values atol=rtol=1e-5 in np.allclose


## 1. Decomposition of a hooke tensor

A Hooke tensor defines a quadratic form on the set of symmetric matrices $\epsilon \in S_d$
$$
    c(\epsilon,\epsilon) = \sum_{ijkl} c_{ijkl} \epsilon_{ij} \epsilon_{kl}.
$$
Note that $S_d$ has dimension $D=d (d+1)/2$. We limit our attention to the case $d=2$, since the case $d=1$ is excessively trivial, and the case $d=3$ would require an implementation of the $6$-dimensional Voronoi reduction, which we do not have at this stage.

We use Selling's decomposition to rewrite this quadratic form as
$$
    c(\epsilon,\epsilon) = \sum_r \rho_r {\rm Tr}(\epsilon m_r)^2
$$
where $\rho_r \geq 0$, $m_r \in S_2(Z)$ is a symmetric matrix with integer coordinates, and $0 \leq r < D (D+1)/2=6$. For that purpose, we rely on Selling's decomposition of the Hooke tensor, which applies in dimension since the linear space of $2\times 2$ symmetric matrices has dimension $3$.

<!--- In the following, we denote as usual $c(\epsilon) := c(\epsilon,\epsilon)$. --->

The stress tensor $\sigma$ depends linearly on the strain tensor $\epsilon$, for a given hooke tensor $c$, and is characterized by the identity
$$
    {\rm Tr}(\sigma \epsilon) = c(\epsilon,\epsilon).
$$
With the correct index conventions, one has $\sigma_{ij} = \sum_{kl} c_{ijkl} \epsilon_{kl}$, or simply $\sigma = c \epsilon$.

### 1.1 Generic tensor

We illustrate the decomposition on a generic tensor, describing the anisotropic elasticity of a mica rock medium, whose layers are rotated.

In [6]:
hooke = mica.extract_xz().rotate_by(0.5)

Selling's decomposition involves $D=6$ weights and offsets, in dimension $d=2$. 

In [7]:
coefs,moffsets = hooke.Selling()

In [8]:
coefs

array([ 4.397544 , 31.388283 ,  3.8697205, 17.653015 , 41.16677  ,
       29.818565 ], dtype=float32)

The offsets are presented as symmetric matrices, with integer entries.

In [9]:
moffsets.shape

(2, 2, 6)

In [10]:
for i in range(6): print(moffsets[...,i],"\n")

[[1 1]
 [1 1]] 

[[-1 -1]
 [-1  0]] 

[[2 1]
 [1 1]] 

[[ 0  0]
 [ 0 -1]] 

[[-1  0]
 [ 0  0]] 

[[1 0]
 [0 1]] 



In order to check the correctness of the decomposition, let us introduce an arbitrary $2\times 2$ symmetric matrix, and evaluate $c(m)$ using the original expression and the decomposition.

In [11]:
m = xp.array(((1.5,2.3),(2.3,3.9)))

In [12]:
sum_hooke = hooke.dot_AA(m) 

In [13]:
mm = fd.as_field(m,coefs.shape)
sum_inner = (coefs*lp.trace(lp.dot_AA(mm,moffsets))**2).sum(axis=0)

In [14]:
assert allclose(sum_hooke,sum_inner)

### 1.2 Isotropic tensor

Isotropic elasticity tensors only have two degrees of freedom. Without loss of generality, we consider the Lamé parameters. These parameters relate the strain tensor $\epsilon$ with the stress tensor $\sigma$
$$
    \sigma = 2 \mu \epsilon + \lambda {\rm Tr}(\epsilon) {\rm Id},
$$
and the quadratic form reads 
$$
    c(\epsilon,\epsilon) = 2 \mu {\rm Tr}(\epsilon^2) + \lambda {\rm Tr}(\epsilon)^2.
$$

In [15]:
hooke = Hooke.from_Lame(xp.array(1.),2.) # xp.array is a type hint for GPU
print(f"""An isotropic hooke tensor :\n{hooke.hooke}\n""")

An isotropic hooke tensor :
[[5. 1. 0.]
 [1. 5. 0.]
 [0. 0. 2.]]



In [16]:
print(f"""Isotropic Hooke tensors are linear combinations of: \n{Hooke.from_Lame(1.,0.).hooke}\n"""
      f"""and \n{Hooke.from_Lame(0.,1.).hooke}\n""")

Isotropic Hooke tensors are linear combinations of: 
[[1. 1. 0.]
 [1. 1. 0.]
 [0. 0. 0.]]
and 
[[2. 0. 0.]
 [0. 2. 0.]
 [0. 0. 1.]]



As their name suggests, isotropic hooke tensors are invariant under rotations.

In [17]:
hooke_rot = copy(hooke).rotate_by(0.5)
assert allclose(hooke.hooke,hooke_rot.hooke)

Selling's decomposition of an isotropic Hooke tensor is very structured and predictable. It involves offsets $m_r$ which are independent of the parameters $(\lambda,\mu)$, and weights $\rho_r(\lambda,\mu)$ depending linearly on the Lame parameters. In addition, several of these coefficients vanish.

In [18]:
coefs,moffsets = hooke.Selling()

In [19]:
coefs

array([-0., -0.,  2.,  4.,  4.,  1.], dtype=float32)

In [20]:
for c,o in zip(coefs,np.moveaxis(moffsets,-1,0)): 
    if c!=0.:
        print(f"coef:{c}, moffset:\n{o}\n")

coef:2.0, moffset:
[[0 1]
 [1 0]]

coef:4.0, moffset:
[[ 0  0]
 [ 0 -1]]

coef:4.0, moffset:
[[-1  0]
 [ 0  0]]

coef:1.0, moffset:
[[1 0]
 [0 1]]



In this special isotropic case, we expect to recover known finite difference schemes.

## 2. Finite difference energy

We approximate the linear elastic energy using a second order accurate finite differences scheme, which exploits our tensor decomposition. The scheme is based on the identity
$$
    c(\epsilon,\epsilon) = \sum_r \rho_r {\rm Tr}(m_r \nabla v)^2
$$
where $(\rho_r,m_r)$ is Selling's decomposition of $c$. We could replace $\sigma$ with $\nabla v$ thanks to the symmetry of $m_r$.

Then we use the finite difference approximations
$$
    {\rm Tr}(m \nabla v) = \sum_{0 \leq i < d} \frac{v_i(x+h s_i m[i] )-v_i(x)}{h s_i} + O(h),
$$
where $s_i\in\{-1,1\}$, $0\leq i < d$ are arbitrary signs, and $m[i]$ denotes the $i$-th column of $m$ (which is a symmetric matrix). Squaring this expression, averaging over all possible sign choices, and summing with weights $\rho_r$, we obtain a second order consistent approximation of the local linear elastic energy.
$$
    c(\epsilon,\epsilon) = \sum_{0 \leq r \leq D (D+1)/2} \frac{\rho_r}{2^d} \sum_{s \in \{-1,1\}^d}  
    \Big(\sum_{0 \leq i < d} \frac{v_i(x+h s_i m_r[i] )-v_i(x)}{h s_i}\Big)^2.
$$

**Remark**
If the coordinates of $m_r[i]$ are not co-prime, for some $0 \leq r < D (D+1)/2$ and $0 \leq i < d$, then one can improve the scheme taking advantage of this fact in the finite differences.

**Scheme order.**
For symmetry reasons, using first order finite differences yields a second order estimation of the elastic energy. Likewise, using second order finite differences yields a fourth order scheme.

In [21]:
def ElasticEnergy(v,hooke,dom,order=1):
    """
    Finite differences approximation of c(ϵ,ϵ), where c is the Hooke tensor and ϵ the stress tensor,
    which is twice the (linearized) elastic energy density.
    """
    assert len(v)==2
    coefs,moffsets = hooke.Selling()
    dvp = tuple( dom.DiffUpwind(v[i], moffsets[i], order=order) for i in range(2))
    dvm = tuple(-dom.DiffUpwind(v[i],-moffsets[i], order=order) for i in range(2))
    
    # Consistent approximations of Tr(moffset*grad(v))
    dv  = ad.array((dvp[0]+dvp[1], dvp[0]+dvm[1], dvm[0]+dvp[1], dvm[0]+dvm[1]))
    dv2 = 0.25* (dv**2).sum(axis=0)
    
    coefs = fd.as_field(coefs,v.shape[1:])
    return (coefs*dv2).sum(axis=0) 

### 2.1 CFL condition

The above discretization of the elastic energy is intended to be used in numerical simulations of the elastic wave equation, which involves a CFL condition. See the [discussion in the one dimension case](Time1D_Div.ipynb). The key to deriving this condition is to upper bound the quadratic form $c(\epsilon,\epsilon)$ in terms of a quadratic form with only diagonal coefficients.

Recall the Cauchy-Schwartz inequality, or arithmetico-geometric inequality
$$
    \Big(\sum_{1 \leq k \leq K} a_k\Big)^2 \leq k \sum_{1 \leq k \leq K} a_k^2,
$$
and observe that $c(\epsilon,\epsilon)$ is a sum of squares, each comprising $K = d (1+r)$ terms, where $r$ is the method's order. Denoting by $A$ the sparse matrix associated with the integrated elastic energy, and by $D$ its diagonal, one therefore has $A \leq K D$.

In the case where the hooke tensors are constant, one may be slightly more explicit, observing that 
$$
    \mathrm{Tr}(c) = \sum_r \rho_r \mathrm{Tr}(m_r^2) \geq \sum_r \rho_r,
$$
since the matrices m_r have integer coefficients (and are non-zero). By $\mathrm{Tr}(c)$ we denote the trace of the Mandel form of $c$, which is also its trace as a quadratic form on the space of $2\times 2$ symmetric matrices equipped with the frobenius inner product.
From this point, one gets $A \leq L_r \mathrm{Tr}(c) \mathrm{Id}$, where $r$ is the method's order, where $L_r = K d c_r$, and $c_2 = 2$ and $c_3 = 1/2^2+2^2+(3/2)^2 = 13/2$ (sum of squares of the finite differences coefficients). Finally, in dimension two, $L_2=16$ and $L_3 = 78$.

In [22]:
hooke = mica.extract_xz().rotate_by(0.5)
coefs,moffsets = hooke.Selling()

In [23]:
tr_decomp = np.sum(coefs*lp.trace(lp.dot_AA(moffsets,moffsets)))
tr_Mandel = lp.trace(hooke.to_Mandel())
assert allclose(tr_decomp,tr_Mandel)

### 2.2 Comparison with automatic differentiation

For comparison, we also evaluate the elastic energy using automatic differentiation for the derivatives of $v$, instead of finite differences, which yields an exact expression. This is only possible when $v$ is provided as a differentiable function, rather than a table.

In [24]:
def ElasticEnergy_ad(v,hooke,X):
    """
    Returns c(ϵ,ϵ), where c is the Hooke tensor, and ϵ is the stress tensor.
    The displacement field must be provided as a function, compatible with automatic differentiation.
    """
    # Differentiate the displacement field
    X_ad = ad.Dense.identity(constant=X,shape_free=(2,))
    grad = v(X_ad).gradient()
    
    # Compute the stress tensor, and the inner product associated with the Hooke tensor.
    ϵ = 0.5*(grad+lp.transpose(grad))
    return hooke.dot_AA(ϵ) 

Let us observe the convergence of the finite element energy toward the exact energy in a smooth periodic setting.

In [25]:
def v(X):
    x0,x1 = X*(2.*np.pi)
    return ad.array((np.cos(x0) - 2.*np.sin(x1),np.cos(x0+2*x1)))

def hooke(X):
    x0,x1 = X*(2.*np.pi)
    angle = 0.3*np.sin(x0)+0.5*np.cos(x1)    
    return mica.extract_xz().rotate_by(angle)

In [26]:
n=20
aX,h = xp.linspace(0,1,n,endpoint=False,retstep=True)
X=ad.array(np.meshgrid(aX,aX,indexing='ij'))
dom = Domain.MockDirichlet(X.shape,h,padding=None) #Periodic domain (wrap instead of pad)

In [27]:
energy_density_fd = ElasticEnergy(   v(X),hooke(X),dom) # Uses samples of v
energy_density_ad = ElasticEnergy_ad(v,   hooke(X),X)   # Uses function v

The finite differences scheme is second order accurate. A fourth order scheme could easily be achieved, in the domain interior, by using higher order finite differences. The treatment of non-periodic boundary conditions in another story altogether, that will be addressed later.

In [28]:
norm_infinity(energy_density_fd-energy_density_ad) / norm_average(energy_density_ad)

array(0.7490726, dtype=float32)

In order to obtain the total elastic energy, the density must be integrated over the domain, not forgetted about the $1/2$ factor.

In [29]:
energy_ad = 0.5 * energy_density_ad.sum() * h**2
energy_fd = 0.5 * energy_density_fd.sum() * h**2
print(f"Relative error: {(energy_ad-energy_fd)/energy_fd}")

Relative error: 0.030708935


### 2.3 Structure of the mass matrix

We evaluate the finite difference scheme on a second order sparse AD vector, to find the mass matrix of the elastic energy. At each point in the domain, the energy density depends on a number of neighbor values of the strain tensor, referred to as the stencil.

In [30]:
#ReloadPackages()

In [31]:
v_sp = ad.Sparse2.identity(constant=np.zeros_like(X))
energy_density_sp = ElasticEnergy(v_sp,hooke(X),dom)
print(f"Stencil cardinality: {energy_density_sp.size_ad2}")

Stencil cardinality: 384


The *simplification* step compresses the sparse matrix by merging entries corresponding to the same coefficient, and removing zero coefficients.
Note that we use `atol=0.` to also remove coefficients arising from cancellation effects: the sum of merged entries vanishes. 
A more careful implementation of `Energy_fd` may allow to bypass these simplification steps.

In [33]:
energy_density_sp.simplify_ad(atol=0.)
print(f"Stencil cardinality: {energy_density_sp.size_ad2}")

CompileException: C:\Users\Shadow\Documents\GitHub\AdaptiveGridDiscretizations\agd\AutomaticDifferentiation\AD_CUDA\../../Eikonal/HFM_CUDA/cuda/NetworkSort.h(290): error: identifier "fls" is undefined
          detected during instantiation of "void variable_length_sort(const T *, int *, int) [with T=IndexT]" 
C:\Users\Shadow\Documents\GitHub\AdaptiveGridDiscretizations\agd\AutomaticDifferentiation\AD_CUDA\cuda\simplify_ad.h(55): here

C:\Users\Shadow\Documents\GitHub\AdaptiveGridDiscretizations\agd\AutomaticDifferentiation\AD_CUDA\../../Eikonal/HFM_CUDA/cuda/NetworkSort.h(291): error: expression must have a constant value
          detected during instantiation of "void variable_length_sort(const T *, int *, int) [with T=IndexT]" 
C:\Users\Shadow\Documents\GitHub\AdaptiveGridDiscretizations\agd\AutomaticDifferentiation\AD_CUDA\cuda\simplify_ad.h(55): here

C:\Users\Shadow\AppData\Local\Temp\tmp6vx72i7i\d2a4ec6216f3b0ed3e97ec963f9b44ce_2.cubin.cu(3): warning: variable "Int_Max" was dec

The energy density at a given point in space is given by a sparse quadratic form, with the above number of non-zero entries at most.

In [ ]:
energy_density_sp[9,5].triplets()

Let us construct the sparse symmetric matrix associated to the total energy.

In [ ]:
energy_sp = 0.5 * energy_density_sp.sum() * h**2
energy_hess = energy_sp.hessian_operator()

Since the finite difference energy is quadratic, it is exactly reproduced by the quadratic form defined by is half hessian.

In [ ]:
v_fl = v(X).flatten()
energy_fl = 0.5*np.dot(v_fl,energy_hess*v_fl)

In [ ]:
# Fails on cupy 6., works on cupy 7.4
assert allclose(energy_fl,energy_fd)

In [ ]:
assert allclose(energy_sp.as_func(v_fl),energy_fd)

The mass matrix based on second order upwind finite differences is (even) less sparse, as could be expected. 

In [ ]:
v_sp = ad.Sparse2.identity(constant=np.zeros_like(X))
energy_density_sp = ElasticEnergy(v_ad,hooke(X),dom,order=2)
print(f"Stencil cardinality: {energy_density_sp.size_ad2}")

In [ ]:
energy_density_sp.simplify_ad(atol=0.)
print(f"Stencil cardinality: {energy_density_sp.size_ad2}")